In [1]:
import sys
import re    # for regular expressions 
import nltk  # for text manipulation 
import string 
import warnings 
import numpy as np 
#import tensorflow as tf
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import sqlite3  as sql

# Plotly visualizations
#from plotly import tools
#import plotly as py
#import plotly.figure_factory as ff
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True) 

# plotly.tools.set_credentials_file(username='AlexanderBach', api_key='o4fx6i1MtEIJQxfWYvU1')


# For oversampling Library (Dealing with Imbalanced Datasets)
#from imblearn.over_sampling import SMOTE
#from collections import Counter

# Other Libraries
import time
from datetime import datetime as dt


%matplotlib inline

In [2]:
pd.set_option("display.max_colwidth", 200) 
pd.set_option('display.max_columns', None)
pd.options.display.width=None
pd.options.display.max_columns = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [256]:
data = pd.read_csv  ( 'IDS.csv' )
#data.info()

In [257]:
#data['Gender'] = data.Gender.str.strip(), 
data.Gender.astype(str)
data['Gender'].unique()

## As data seems to be clean for Gender column, no changes required

array(['Female', 'Male'], dtype=object)

In [258]:
## Colums are Gender & Programming_Rate are good
import datetime
#data[data['Age'].isnull()]['Age'] =
#data[data['Age'].isnull()] = dt.today() - data[data['Age'].isnull()]['DOB'].dt.date
#dt.today() - pd.to_datetime ( data[data['Age'].isnull()]['DOB'])
#pd.date_range(start=pd.to_datetime ( data[data['Age'].isnull()]['DOB']),end=dt.today(),freq='Y')

#data[data['Age'].isnull()]['Age'] = datetime.date.today().year - pd.DatetimeIndex ( data[data['Age'].isnull()]['DOB']).year

data.loc [data['Age'].isnull() ,'Age'] = datetime.date.today().year - pd.DatetimeIndex ( data[data['Age'].isnull()]['DOB']).year

## Age columns fixed with no null values 

In [259]:
data.Age

0      19.0
1      19.0
2      19.0
3      18.0
4      19.0
5      19.0
6      19.0
7      19.0
8      18.0
9      19.0
10     18.0
11     18.0
12     18.0
13     18.0
14     19.0
15     19.0
16     18.0
17     19.0
18     18.0
19     18.0
20     18.0
21     18.0
22     19.0
23     19.0
24     20.0
25     19.0
26     19.0
27     19.0
28     19.0
29     20.0
       ... 
280    18.0
281    21.0
282    19.0
283    20.0
284    18.0
285    18.0
286    19.0
287    18.0
288    19.0
289    18.0
290    19.0
291    18.0
292    19.0
293    19.0
294    20.0
295    18.0
296    19.0
297    19.0
298    17.0
299    17.0
300    18.0
301    20.0
302    19.0
303    18.0
304    18.0
305    19.0
306    19.0
307    19.0
308    19.0
309    20.0
Name: Age, Length: 310, dtype: float64

In [264]:
data['Weight'] = data.Weight.str.replace ( '😑','0').str.replace('Lol','0').str.replace('kgs','').str.replace('kg','').str.replace(' ','')
data.loc[data.Weight.isnull(),'Weight'] = 0



In [261]:
data.loc[data.Weight.str.contains('billion') == True,'Weight'] = float ( data[data.Weight.str.contains('billion') == True]['Weight'].str.replace(' ','').str.replace('billiontonne','') )* 1000000000 * 907.18474

In [266]:
data.Weight.astype(float)

0      74.0
1      62.0
2      62.0
3      60.0
4      56.0
5      79.0
6      58.0
7      63.0
8      65.0
9      60.0
10     51.0
11     64.0
12     69.0
13     57.0
14     70.0
15     49.0
16     40.0
17     58.2
18     61.0
19     60.0
20     67.0
21     80.0
22     70.0
23     88.0
24     58.0
25     57.5
26     54.0
27     46.0
28     55.0
29     63.0
       ... 
280    87.5
281    73.0
282    55.0
283    45.0
284    52.0
285    67.0
286    53.0
287    90.8
288    48.5
289    43.0
290    52.0
291    70.0
292    57.0
293    80.0
294    60.0
295    65.0
296    59.0
297    59.0
298    49.0
299    49.0
300    51.0
301    68.0
302    50.0
303    45.0
304    58.0
305    65.0
306    65.0
307    65.0
308    56.0
309    58.0
Name: Weight, Length: 310, dtype: float64

In [288]:
# 1feet = 12 inches
# 1inch = 2.54 cm

def trans ( x):
    if 'mm' in x:
        return int ( x.replace('mm','') ) / 10
    elif "'" in x:
        i , j =  x.replace ( "'",'.').replace('"','').split ( '.')
        return ( int ( i ) *12 *2.54 + int(j)*2.54 )
    else:
        return x

data['Height'] = data.Height.str.replace('cm','').str.replace('😂','0').str.replace('Lol','0').str.replace(' ','').apply (trans).astype(float)



In [291]:
data.Height

0      160.02
1      176.00
2      176.00
3      145.00
4      185.00
5      170.00
6      168.00
7      156.00
8      158.00
9      180.34
10     150.00
11     165.10
12     193.00
13     183.00
14     175.00
15     175.00
16     156.00
17     180.00
18       5.40
19     178.00
20     180.00
21     160.00
22     170.00
23     179.00
24     182.80
25     179.00
26     180.00
27     156.00
28     170.00
29     168.00
        ...  
280    186.00
281    177.00
282    165.00
283    155.00
284    170.00
285    174.00
286    160.00
287    192.00
288    158.00
289    159.00
290    177.00
291    186.00
292    170.00
293    175.00
294    173.00
295    175.00
296    186.00
297    186.00
298    170.00
299    170.00
300    160.00
301    167.00
302    158.00
303    160.00
304    160.00
305    180.00
306    180.00
307    180.00
308    159.00
309    169.00
Name: Height, Length: 310, dtype: float64

In [299]:
data['DOB'] = pd.DatetimeIndex ( data.DOB )

In [301]:
data.DOB


0     2000-07-30
1     2000-06-04
2     2000-06-04
3     2000-02-21
4     2000-08-15
5     1999-06-21
6     2000-06-03
7     1999-10-25
8     2000-10-12
9     2000-07-10
10    2000-04-13
11    2000-09-10
12    2018-01-20
13    2000-09-06
14    2000-01-08
15    1999-12-23
16    2000-11-25
17    2000-04-01
18    2000-12-14
19    2000-01-07
20    2000-02-14
21    2000-11-25
22    2000-05-21
23    2000-01-07
24    2000-06-10
25    2000-07-13
26    2000-04-19
27    1999-12-24
28    2000-07-28
29    1999-10-08
         ...    
280   2001-04-16
281   1999-06-18
282   2000-07-12
283   1999-08-17
284   2001-01-09
285   2001-03-05
286   2000-06-05
287   2000-10-06
288   2000-03-07
289   2000-10-23
290   2000-03-29
291   2001-07-09
292   1999-08-29
293   2000-06-05
294   2000-03-26
295   2001-04-10
296   2000-02-02
297   2000-02-02
298   2019-08-04
299   2019-08-04
300   2000-11-24
301   1999-08-05
302   1999-10-06
303   2001-01-06
304   2019-09-08
305   2000-01-19
306   2000-01-19
307   2000-01-

In [316]:
data['Sim_Course'] = data.Sim_Course.apply ( lambda x: False if x == 'No' else True )
data.Sim_Course

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
280    True
281    True
282    True
283    True
284    True
285    True
286    True
287    True
288    True
289    True
290    True
291    True
292    True
293    True
294    True
295    True
296    True
297    True
298    True
299    True
300    True
301    True
302    True
303    True
304    True
305    True
306    True
307    True
308    True
309    True
Name: Sim_Course, Length: 310, dtype: bool

In [317]:
data.head()

,#,Gender,Height,Weight,DOB,Age,Sun_Sign,CGPA,Programming_Rate,Preferred_Lang,Sim_Course,Focus,DS_Same_As_DA,Mean,Median,Sleeping_Time,Sleep_Duration,Like_Maths,Engg_First_Choice,Inst_Choice,Physical_Activity
0,1,Female,160.02,74,2000-07-30,19.0,Aquarius,8.29,3,C++,True,Data,No,6.0,8,11:00:00 PM,5,Yes,Yes,Doesn't matter,On daily basis
1,2,Male,176.00,62,2000-06-04,19.0,Gemini,8.27,4,C,True,Data,May be,6.0,9,1:00:00 AM,6.5,No,Yes,Doesn't matter,Once in a week
2,3,Male,176.00,62,2000-06-04,19.0,Gemini,8.27,4,C,True,Data,May be,6.0,9,1:00:00 AM,6.5,No,Yes,Doesn't matter,Once in a week
3,4,Female,145.00,60,2000-02-21,18.0,Don't Know,9.21,3,Python,True,Data,No,6.0,9,11:00:00 AM,6,Yes,Yes,Doesn't matter,Once in a week
4,5,Male,185.00,56,2000-08-15,19.0,Leo,8.46,4,C,True,Data,No,6.0,8,11:30:00 PM,8,Yes,Yes,Doesn't matter,Once in a week


In [320]:
data.DS_Same_As_DA.unique()

array(['No', 'May be', 'Yes'], dtype=object)

In [326]:
data.Physical_Activity.unique()

array(['On daily basis', 'Once in a week', 'Not so often', 'Never',
       'Once in a month', nan], dtype=object)

In [327]:
data.Inst_Choice.unique()

array(["Doesn't matter", 'Female', 'Male'], dtype=object)